In [58]:
import pandas as pd

# Load the SMT Output CSV file into a DataFrame

#Make sure times have been converted in EXCEL to h:mm:ss format

df_csv = pd.read_csv(r"C:\Users\12524\Desktop\Research Data\7.15.24 SMT Localization Data\100m5MHz7.15.24.csv")

# Display the first few rows of the DataFrame to confirm it loaded correctly
df_csv.head()

# Display the columns to confirm structure
#print("CSV File Columns:")
#print(df_csv.columns)

,Meas ID,Algorithm,Center Frequency,Sample Rate,Num Samples,Latitude,Longitude,Elevation,RHO,CEP,Total Sensors,Valid Sensors,Overloaded Sensors,Sensor Names,Time,Comment
0,535,com.keysight.ratel.tdoa,3320000000,6400000,32000,0.0,0.0,0,0.020610,0,4,4,0,192.168.21.13; 192.168.21.14; 192.168.21.15; 1...,19:04:23,NaN
1,536,com.keysight.ratel.tdoa,3320000000,6400000,32000,0.0,0.0,0,0.020994,0,4,4,0,192.168.21.13; 192.168.21.14; 192.168.21.15; 1...,19:21:38,NaN
2,537,com.keysight.ratel.tdoa,3320000000,6400000,32000,0.0,0.0,0,0.022034,0,4,4,0,192.168.21.13; 192.168.21.14; 192.168.21.15; 1...,19:21:41,NaN
3,538,com.keysight.ratel.tdoa,3320000000,6400000,32000,0.0,0.0,0,0.020742,0,4,4,0,192.168.21.13; 192.168.21.14; 192.168.21.15; 1...,19:21:45,NaN
4,539,com.keysight.ratel.tdoa,3320000000,6400000,32000,0.0,0.0,0,0.020301,0,4,4,0,192.168.21.13; 192.168.21.14; 192.168.21.15; 1...,19:21:48,NaN


In [59]:
# Load the vehicleOut TXT file into a DataFrame
# Load the TXT file without headers, split columns
column_names = ['id', 'GTLongitude', 'GTLatitude', 'GTAltitude', 'extra1', 'extra2', 'extra3', 'time', 'extra4', 'extra5']
df_txt = pd.read_csv(r"C:\Users\12524\Desktop\Research Data\7.15.24 SMT Localization Data\colejuly15\root\Results\2024-07-15_15_21_32_vehicleOut.txt", 
                     delimiter=',', header=None, names=column_names)
print(df_txt.head())

#print("\nTXT File Columns:")
#print(df_txt.columns)

   id  GTLongitude  GTLatitude  GTAltitude  \
0   1   -78.696248   35.727483       0.468   
1   2   -78.696248   35.727483       0.464   
2   3   -78.696248   35.727483       0.481   
3   4   -78.696248   35.727483       0.511   
4   5   -78.696248   35.727483       0.539   

                                              extra1            extra2  \
0  (-0.004192729014903307,0.1358097493648529,-0.1...   (0.0,0.0,-0.02)   
1  (-0.004284991417080164,0.14045703411102295,-0....   (0.0,0.0,-0.01)   
2  (-0.004544282332062721,0.14492923021316528,-0....  (0.0,0.01,-0.02)   
3  (-0.004504253156483173,0.1490633338689804,-0.1...   (0.0,0.0,-0.03)   
4  (-0.00461655342951417,0.15312807261943817,-0.1...   (0.0,0.0,-0.03)   

   extra3                        time  extra4  extra5  
0  49.446  2024-07-15 15:21:38.404292       4      27  
1  49.451  2024-07-15 15:21:39.407355       4      27  
2  49.457  2024-07-15 15:21:40.410652       4      27  
3  49.461  2024-07-15 15:21:41.414105       4      27 

In [60]:
from datetime import datetime, timedelta

# Convert the 'Time' column in CSV to datetime format and subtract four hours
df_csv['Time'] = pd.to_datetime(df_csv['Time'], format='%H:%M:%S').apply(lambda x: (datetime.combine(datetime.today(), x.time()) - timedelta(hours=4)).time())

# Convert the 'time' column in TXT to datetime format
df_txt['time'] = pd.to_datetime(df_txt['time'])

# Display the first few rows to confirm conversion
print("CSV File with Adjusted Time Column:")
print(df_csv.head())

print("\nTXT File with Converted Time Column:")
print(df_txt.head())

CSV File with Adjusted Time Column:
   Meas ID                Algorithm  Center Frequency  Sample Rate  \
0      535  com.keysight.ratel.tdoa        3320000000      6400000   
1      536  com.keysight.ratel.tdoa        3320000000      6400000   
2      537  com.keysight.ratel.tdoa        3320000000      6400000   
3      538  com.keysight.ratel.tdoa        3320000000      6400000   
4      539  com.keysight.ratel.tdoa        3320000000      6400000   

   Num Samples  Latitude  Longitude  Elevation       RHO  CEP  Total Sensors  \
0        32000       0.0        0.0          0  0.020610    0              4   
1        32000       0.0        0.0          0  0.020994    0              4   
2        32000       0.0        0.0          0  0.022034    0              4   
3        32000       0.0        0.0          0  0.020742    0              4   
4        32000       0.0        0.0          0  0.020301    0              4   

   Valid Sensors  Overloaded Sensors  \
0              4      

In [61]:
from datetime import datetime

# Helper function to convert time to total seconds since midnight
def time_to_seconds(time_obj):
    return time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second

# Function to find the closest timestamp
def find_closest_time(row, df_txt):
    target_time_seconds = time_to_seconds(row['Time'])
    df_txt['time_only'] = df_txt['time'].dt.time
    df_txt['time_seconds'] = df_txt['time_only'].apply(time_to_seconds)
    closest_row = df_txt.iloc[(df_txt['time_seconds'] - target_time_seconds).abs().argsort()[:1]]
    return closest_row.iloc[0]['GTLongitude'], closest_row.iloc[0]['GTLatitude'], closest_row.iloc[0]['GTAltitude']

# Apply the function to each row in the CSV
df_csv[['GTLongitude', 'GTLatitude', 'GTAltitude']] = df_csv.apply(lambda row: pd.Series(find_closest_time(row, df_txt)), axis=1)

# Display the updated CSV DataFrame
print("Updated CSV File with Closest Coordinates:")
print(df_csv.head())


Updated CSV File with Closest Coordinates:
   Meas ID                Algorithm  Center Frequency  Sample Rate  \
0      535  com.keysight.ratel.tdoa        3320000000      6400000   
1      536  com.keysight.ratel.tdoa        3320000000      6400000   
2      537  com.keysight.ratel.tdoa        3320000000      6400000   
3      538  com.keysight.ratel.tdoa        3320000000      6400000   
4      539  com.keysight.ratel.tdoa        3320000000      6400000   

   Num Samples  Latitude  Longitude  Elevation       RHO  CEP  Total Sensors  \
0        32000       0.0        0.0          0  0.020610    0              4   
1        32000       0.0        0.0          0  0.020994    0              4   
2        32000       0.0        0.0          0  0.022034    0              4   
3        32000       0.0        0.0          0  0.020742    0              4   
4        32000       0.0        0.0          0  0.020301    0              4   

   Valid Sensors  Overloaded Sensors  \
0              

In [62]:
# Define the path where you want to save the updated CSV file
output_csv_path = r'C:\Users\12524\Desktop\Research Data\7.15.24 SMT Localization Data\updated_100m5MHz7.15.24.csv'

# Save the DataFrame to CSV
df_csv.to_csv(output_csv_path, index=False)

print(f"Updated CSV file has been saved to {output_csv_path}")


Updated CSV file has been saved to C:\Users\12524\Desktop\Research Data\7.15.24 SMT Localization Data\updated_100m5MHz7.15.24.csv
